In [1]:
from sympy import *
import numpy as np
import k3d
from k3d import transform
import time
import keyboard
import ipywidgets as widgets
from IPython.display import clear_output
def mateq2list(lhs: Matrix, rhs: Matrix):
    L = []
    for i in range(lhs.rows):
        L.append(Eq(lhs[i], rhs[i], evaluate=False))
    return L

In [2]:
## Variables
t = symbols("t")                                                # Tiempo
g = symbols("g")                                                # Constante gravitacional
m = symbols("m")                                                # Masa del cohete
T = symbols("T", cls=Function)                                  # Empuje de propulsor
T_a1, T_a2 = symbols("T_theta, T_phi", cls=Function)            # Orientación de propulsor
a1, a2, a3 = symbols("phi, theta, psi", cls=Function)           # Orientación del cohete
vx, vy, vz = symbols("v_x, v_y, v_z", cls=Function)             # Velocidad

## Ecuaciones del movimiento en traslación
# Fuerzas Body frame
F_B = Matrix([
    T(t)*cos(T_a1(t)),
    T(t)*sin(T_a1(t))*cos(T_a2(t)),
    T(t)*sin(T_a1(t))*sin(T_a2(t))
])

# Matriz transformación de Body frame a Inertial frame
Q_BI = Matrix([
    [cos(a1(t))*cos(a2(t)), cos(a1(t))*sin(a2(t))*sin(a3(t))-sin(a1(t))*sin(a3(t)), cos(a1(t))*sin(a2(t))*cos(a3(t))+sin(a1(t))*sin(a3(t))],
    [sin(a1(t))*cos(a2(t)), sin(a1(t))*sin(a2(t))*sin(a3(t))+sin(a1(t))*cos(a3(t)), sin(a1(t))*sin(a2(t))*cos(a3(t))-cos(a1(t))*sin(a3(t))],
    [-sin(a2(t)), cos(a2(t))*sin(a3(t)), cos(a2(t))*cos(a3(t))]
])

# Segunda ley de newton
G = Matrix([0, 0, -g])
V = Matrix([vx(t), vy(t), vz(t)])
eqs_1 = mateq2list(diff(V, t), G + Q_BI*F_B/m)
for eq in eqs_1: display(eq)

Eq(Derivative(v_x(t), t), ((-sin(phi(t))*sin(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + T(t)*cos(T_theta(t))*cos(phi(t))*cos(theta(t)))/m)

Eq(Derivative(v_y(t), t), ((sin(phi(t))*sin(psi(t))*sin(theta(t)) + sin(phi(t))*cos(psi(t)))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + T(t)*sin(phi(t))*cos(T_theta(t))*cos(theta(t)))/m)

Eq(Derivative(v_z(t), t), -g + (T(t)*sin(T_phi(t))*sin(T_theta(t))*cos(psi(t))*cos(theta(t)) + T(t)*sin(T_theta(t))*sin(psi(t))*cos(T_phi(t))*cos(theta(t)) - T(t)*sin(theta(t))*cos(T_theta(t)))/m)

In [3]:
## Variables
l = symbols("l")                                                # Largo del cohete
Ix, Iy, Iz = symbols("I_x, I_y, I_z")                           # Momentos de inercia principales
wx, wy, wz = symbols("omega_x, omega_y, omega_z", cls=Function) # Velocidad angular Body frame

## Ecuaciones del movimiento en rotación
# Momentos Body frame
T_R = Matrix([-l/2, 0, 0])  # Posición de propulsor respecto a centro de masa
M_B = T_R.cross(F_B)

# Ecuaciónes de euler para rotación Body frame
I = diag(Ix, Iy, Iz)
w = Matrix([wx(t), wy(t), wz(t)])
eqs_2 = mateq2list(diff(w, t), I**-1*(M_B-w.cross(I*w)))
for eq in eqs_2: display(eq)

# Convertir velocidad angular Body frame a Inertial frame
W = Matrix([a1(t), a2(t), a3(t)]).diff(t)
Qw = Matrix([
    [1, sin(a1(t))*tan(a2(t)), cos(a1(t))*tan(a2(t))],
    [0, cos(a1(t)), -sin(a1(t))],
    [0, sin(a1(t))/cos(a2(t)), cos(a1(t))/cos(a2(t))]
])
eqs_3 = mateq2list(W, Qw*w)
for eq in eqs_3: display(eq)

Eq(Derivative(omega_x(t), t), (I_y*omega_y(t)*omega_z(t) - I_z*omega_y(t)*omega_z(t))/I_x)

Eq(Derivative(omega_y(t), t), (-I_x*omega_x(t)*omega_z(t) + I_z*omega_x(t)*omega_z(t) + l*T(t)*sin(T_phi(t))*sin(T_theta(t))/2)/I_y)

Eq(Derivative(omega_z(t), t), (I_x*omega_x(t)*omega_y(t) - I_y*omega_x(t)*omega_y(t) - l*T(t)*sin(T_theta(t))*cos(T_phi(t))/2)/I_z)

Eq(Derivative(phi(t), t), omega_x(t) + omega_y(t)*sin(phi(t))*tan(theta(t)) + omega_z(t)*cos(phi(t))*tan(theta(t)))

Eq(Derivative(theta(t), t), omega_y(t)*cos(phi(t)) - omega_z(t)*sin(phi(t)))

Eq(Derivative(psi(t), t), omega_y(t)*sin(phi(t))/cos(theta(t)) + omega_z(t)*cos(phi(t))/cos(theta(t)))

In [4]:
## Variables
x, y, z = symbols("x, y, z", cls=Function)  # Posición

## Sistema de ecuaciones completo
R = Matrix([x(t), y(t), z(t)])
eqs_4 = mateq2list(diff(R, t), V)
eqs = eqs_4 + eqs_1 + eqs_2 + eqs_3
eqs_rhs = []
for eq in eqs:
    display(eq)
    eqs_rhs.append(eq.rhs)

Eq(Derivative(x(t), t), v_x(t))

Eq(Derivative(y(t), t), v_y(t))

Eq(Derivative(z(t), t), v_z(t))

Eq(Derivative(v_x(t), t), ((-sin(phi(t))*sin(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + T(t)*cos(T_theta(t))*cos(phi(t))*cos(theta(t)))/m)

Eq(Derivative(v_y(t), t), ((sin(phi(t))*sin(psi(t))*sin(theta(t)) + sin(phi(t))*cos(psi(t)))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + T(t)*sin(phi(t))*cos(T_theta(t))*cos(theta(t)))/m)

...

Eq(Derivative(omega_y(t), t), (-I_x*omega_x(t)*omega_z(t) + I_z*omega_x(t)*omega_z(t) + l*T(t)*sin(T_phi(t))*sin(T_theta(t))/2)/I_y)

Eq(Derivative(omega_z(t), t), (I_x*omega_x(t)*omega_y(t) - I_y*omega_x(t)*omega_y(t) - l*T(t)*sin(T_theta(t))*cos(T_phi(t))/2)/I_z)

Eq(Derivative(phi(t), t), omega_x(t) + omega_y(t)*sin(phi(t))*tan(theta(t)) + omega_z(t)*cos(phi(t))*tan(theta(t)))

Eq(Derivative(theta(t), t), omega_y(t)*cos(phi(t)) - omega_z(t)*sin(phi(t)))

Eq(Derivative(psi(t), t), omega_y(t)*sin(phi(t))/cos(theta(t)) + omega_z(t)*cos(phi(t))/cos(theta(t)))

In [5]:
from numpy import *

## Sistema de ecuaciones evaluable numéricamente
sys = lambdify(([x(t), y(t), z(t),
                vx(t), vy(t), vz(t),
                wx(t), wy(t), wz(t),
                a1(t), a2(t), a3(t)],       # Vector estado
                [T(t), T_a1(t), T_a2(t)],   # Vector propulsor
                [m, g, l, Ix, Iy, Iz]),     # Vector parametros
                eqs_rhs)

In [48]:
# Vector estado
#              x  y  z vx vy vz wx wy wz ya pi ro
ss = np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0])

# Vector control de propulsor
#              T a1 a2
tt = np.array([1, 0, 0])

# Vector parametros
#                m  g  l   Ix   Iy   Iz
pam = np.array([10, 0, 1, 0.3, 0.3, 0.3])

## Simulación con metodo de Euler (flashbacks de sistemas de control)
frog = k3d.stl(
    open("Missile Toad.stl", "rb").read(),
    color=0x00ff00,
    transform=transform(
        scaling=[0.01,0.01,0.01]
    )
)

vec_front = k3d.vectors([0, 0, 0], [0, 0, 1], color=0x00ff00)

plot = k3d.plot()
plot += frog
plot += vec_front
plot.display()

def update_plot():
    frog.transform.translation = ss[0:3]
    vec_front.origins = ss[0:3]
    pitch = ss[10]
    yaw = ss[9]
    x = cos(yaw)*cos(pitch)
    y = sin(yaw)*cos(pitch)
    z = sin(-pitch)

    vec_front.vectors = [x, y, z]
    vec_front.send_state()

    a = np.arccos(np.dot([x, y, z], [0, 0, 1]) / np.linalg.norm([x, y, z]))
    frog.transform.rotation = np.concatenate(([a], np.cross([0, 0, 1], [x, y, z])))

h = 1/30
lt = time.time()
while True:
    if keyboard.is_pressed("esc"): break
    ct = time.time()
    if ct > lt + h:
        ss = ss + [Y*h for Y in sys(ss, tt, pam)]
        update_plot()
        time.sleep(h)


Output()

In [10]:
np.concatenate

array([ 0., -1.,  0.], dtype=float32)